In [6]:
#-----------------
# VERSION
# 07-12-2020
#-----------------

# For webpage Scrapper
import pandas as pd 
import requests
from bs4 import BeautifulSoup

# For Google Sheet functions
import pickle
import os.path
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


# -------------------------------------------------
# File evaluation - Roberto Schlatter
#-------------------------------------------------
   
sFilename = 'transactions-from-01012010-to-09122020.csv'

df_transactions = pd.read_csv(sFilename, sep=';', encoding= 'unicode_escape')

lst_header = ['Transaktionen', 'Name']

#----------
# DIVIDENDE
#----------
# Filter table to show only columns "Transaktionen", "Name"
df_dividende = df_transactions[df_transactions['Transaktionen'].isin(['Dividende'])]

df_dividende['Nettobetrag in der Währung des Kontos'] = pd.to_numeric(df_dividende['Nettobetrag in der Währung des Kontos'])

# Final dataframe for dividends
lst_header_div = ['Symbol']
df_export_dividende = df_dividende.groupby(lst_header_div)['Nettobetrag in der Währung des Kontos'].sum()

#Nach Datum sortiert
lst_header_div = ['Datum', 'Symbol']
df_export_dividende_2 = df_dividende.groupby(lst_header_div)['Nettobetrag in der Währung des Kontos'].sum()
print(df_export_dividende_2)
#df_export_dividende.to_excel('nettodividende.xlsx')
#print("Dividende: ", df_export_dividende)

#-------
# KOSTEN
#-------
# KAUF & VERKAUF Kosten
df_kosten_total = df_transactions[df_transactions['Transaktionen'].isin(['Kauf', 'Verkauf'])]

# Final dataframe for costs
df_export_kosten_total = df_kosten_total.groupby(lst_header)['Kosten'].sum()

print(df_export_kosten_total)

df_export_kosten_total.to_excel('kosten.xlsx')

#----------------------------------------------
# Transaktionen - total performance Berechnung
#----------------------------------------------
df_export_performance = df_transactions[df_transactions['Transaktionen'].isin(['Kauf', 'Verkauf'])]

df_export_performance = df_export_performance[['Datum', 'Transaktionen', 'Symbol', 'Name', 'Anzahl', 'Stückpreis', 'Kosten', 'Nettobetrag in der Währung des Kontos', 'Währung Nettobetrag']]

df_export_performance = df_export_performance.sort_values(by=['Symbol','Transaktionen','Datum'])

#----------------------------------------
# Google Sheets Storing - STOCK-MONITOR
#----------------------------------------

def storeDataToGoogleSheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME, df_gold, sOrientation):
    def Create_Service(client_secret_file, api_service_name, api_version, *scopes):
        global service
        SCOPES = [scope for scope in scopes[0]]
        #print(SCOPES)
        cred = None

        if os.path.exists('token_write.pickle'):
            with open('token_write.pickle', 'rb') as token:
                cred = pickle.load(token)

        if not cred or not cred.valid:
            if cred and cred.expired and cred.refresh_token:
                cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
                cred = flow.run_local_server()

            with open('token_write.pickle', 'wb') as token:
                pickle.dump(cred, token)

        try:
            service = build(api_service_name, api_version, credentials=cred)
            print(api_service_name, 'service created successfully')
            #return service
        except Exception as e:
            print(e)
            #return None
        
    # change 'my_json_file.json' by your downloaded JSON file: my_credentials.json
    Create_Service('my_credentials.json', 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])
    
    def Export_Data_To_Sheets():
        response_date = service.spreadsheets().values().update(
            spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
            valueInputOption='RAW',
            range=SAMPLE_RANGE_NAME,
            body=dict(
                #Orientation: ROWS oder COLUMNS
                majorDimension=sOrientation,
                values=df_gold.T.reset_index().T.values.tolist())
        ).execute()
        print('Sheet successfully Updated')

    Export_Data_To_Sheets()
    print(df_export_dividende)
 #call storeDataToGoogleSheet function
storeDataToGoogleSheet('1RX2lJwZYWUO_sJUME1M9CsMFkGQNEqMCiY2Y6vCZjF0', 'SUMMARY!B53', df_export_dividende, 'COLUMNS')
storeDataToGoogleSheet('1RX2lJwZYWUO_sJUME1M9CsMFkGQNEqMCiY2Y6vCZjF0', 'HISTORIC-DATA!A03', df_export_kosten_total, 'COLUMNS')
storeDataToGoogleSheet('1RX2lJwZYWUO_sJUME1M9CsMFkGQNEqMCiY2Y6vCZjF0', 'HISTORIC-DATA!Z03', df_export_dividende_2, 'COLUMNS')
storeDataToGoogleSheet('1RX2lJwZYWUO_sJUME1M9CsMFkGQNEqMCiY2Y6vCZjF0', 'HISTORIC-DATA!J02', df_export_performance, 'ROWS')


Datum                Symbol
04-01-2016 11:30:53  SPY5       21.36
04-04-2016 11:06:08  SPY5       22.14
05-04-2019 14:17:25  TLT         9.73
05-06-2020 14:48:26  TLT         2.50
05-09-2018 13:51:49  SMMCHA     15.01
                                ...  
30-09-2020 11:41:24  SMMCHA    273.13
30-09-2020 13:52:38  VTI        57.57
30-12-2019 13:58:20  VWRL       16.35
30-12-2019 13:59:35  VFEM       10.33
30-12-2019 15:02:27  VTI        47.11
Name: Nettobetrag in der Währung des Kontos, Length: 76, dtype: float64
Transaktionen  Name                          
Kauf           AC IMMUNE SA ORD                   26.23
               ACTELION N                         40.75
               ALIBABA GROUP HOLDING ADR REP     111.79
               ALPHABET CL C ORD                 113.00
               AMAZON COM ORD                    115.41
                                                  ...  
Verkauf        WASTE MANAGEMENT ORD                8.37
               X MSCI CHINA                 